In [53]:
#Player stats
# Get all player stats for each year
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, playercareerstats, playercareerstats, playergamelogs

import numpy as np
import pandas as pd


In [58]:
# TODO Find efficient weight paramater (ELO) to be used in linear regression
# TODO plot data, linear regression may not capture much data due to distribition
# TODO plot player and game data with various plots (list of plots on note)

In [55]:
year_start = 1946
file_name = "season_year_start_player_data_"
results = []
count = 0
for year_addition in range(len(results)):
    results[count].to_csv(file_name + str(year_start + year_addition))
    print(count)
    count = count + 1
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


In [56]:
players = pd.DataFrame(pd.read_csv("players.csv"))

def find_player_id(player_full_name):
    for i in range(0, len(players)):
        player = players.iloc[i]
        if player["full_name"] == player_full_name:
            return player['id']
        

        
def get_player_stat_for_season(player_id,  season_start, player_name = None):
    # check if player played for that season.
    # if not played return program with error
    file_location = "../player_season/player_season_data/season_year_start_player_data_" + str(season_start)
    season = pd.read_csv(file_location)
    return(season)



def get_stat_for_player(season, player_id):
     player_stat = season[season['PLAYER_ID'] == player_id]        
     return player_stat
             
# data frame object
# player_season_stat = get_stat_for_player(get_player_stat_for_season(player_id=find_player_id("LeBron James"), season_start="2014"),player_id=find_player_id("LeBron James"))


get_stat_for_player(get_player_stat_for_season(player_id=find_player_id("LeBron James"), season_start="2014"),player_id=find_player_id("LeBron James")).to_csv("lebron_test_file.csv")







In [57]:
# data preprocessing and cleaning and analysis

# for every player_stat_object 